 This notebook has an approach that failed to give any satisfying results. It was highly inspired from PyTorch's tutorial https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html. Basically network converged very fast to just rewriting the last input. I was not sure why it was the case. I was using batches of size 1 here, and kind of very sequential learning as they did in the tutorial. Probably it lead to failure. Also very big resolution which I've used there didn't help<br>
 Warning, this notebook is quite messy

In [66]:
%load_ext autoreload
%autoreload 2

from midiToTxt import converter
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from tqdm import tqdm
from torch import optim
import pypianoroll
import random

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
START_TOKEN = "<START>"
END_TOKEN = "<END>"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [83]:
class midiTextGeneratorDataset(Dataset):
    def __init__(self, corpus:str):
        self.unique_letters = set(corpus)
        self.unique_letters.remove("\n")
        
        self.unique_letters.add(START_TOKEN)
        self.unique_letters.add(END_TOKEN)
        
        self.num2word = self.get_num2word_mapping(self.unique_letters)
        self.word2num = {v:k for k,v in self.num2word.items()}
        
        self.tensor_size = len(self.word2num)
        
        self.data = corpus.split("\n")
    
    def get_num2word_mapping(self, letters: set) -> dict:
        mapping = {}
        for index, l in enumerate(letters):
            mapping[index] = l
            
        return mapping
    
    def token_to_tensor(self, token: str) -> torch.tensor:
        tensor = torch.zeros((self.tensor_size, 1))
        if token == START_TOKEN:
            tensor[self.word2num[START_TOKEN]] = 1
        
        elif token == END_TOKEN:
            tensor[self.word2num[END_TOKEN]] = 1
        
        else:  
            for letter in token:
                tensor[self.word2num[letter]] = 1
        
        return torch.unsqueeze(tensor.view(1, -1),0) # we want [1, 1, num_of_characters]
    
    def tensor_to_token(self, tensor: torch.tensor) -> str:
        notes = tensor.nonzero()
        token = ""
        for _, note in notes:
            token += self.num2word[note.item()]
            if len(token) > 10:
                break
            
        return token
    
    def __len__(self) -> int:
        return len(self.data)
    
    def __getitem__(self, idx):
        song = self.data[idx]
        song = song.split(" ")
        song.insert(0, START_TOKEN)
        song.append(END_TOKEN)
        
        inp = self.token_to_tensor(song[0])
        #print(inp.shape)
        for token in song[1:]:
            t = self.token_to_tensor(token)
            #print(t.shape)
            inp = torch.cat((inp, t), dim=0)
        
        return inp

    
        

In [4]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.gru = nn.GRU(output_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, input, hidden):
        input = input.view(1,1,-1)
        output, hidden = self.gru(input, hidden)
        #print(output.shape)
        output = self.out(output[0])
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=DEVICE)

In [5]:
def train(input, decoder, decoder_optimizer, criterion):
    #Input already contains start and end tokens!
    decoder_optimizer.zero_grad()
    
    target_length = input.shape[0]
    #print(target_length)
    loss = 0
    
    teacher_forcing = True
    decoder_input = input[0] #Taking START token out
    decoder_hidden = decoder.initHidden()
    for i in range(1, target_length):
        decoder_output, decoder_hidden = decoder(
            decoder_input, decoder_hidden)
        
        loss += criterion(decoder_output, input[i]) # Based on i-1 chord does model predicted ith chord?
        #print(loss)
        decoder_input = input[i] # i ths chord is input to the next iteration
        
    loss.backward()
    
    decoder_optimizer.step()

    return loss.item() / target_length

In [6]:
def train_epoch(dataloader, decoder, decoder_optimizer, criterion):
    loop = tqdm(dataloader, leave=True)

    for batch_idx, (input) in enumerate(loop):
        input = input.to(DEVICE)
        #print(input.shape)
        loss = train(input[0], decoder, decoder_optimizer, criterion)
        # update tqdm loop
        loop.set_postfix(loss=loss)

In [7]:
path = "lstm_dataset.txt"

with open(path, "r") as f:
    corpus = f.read()

In [8]:
dataset = midiTextGeneratorDataset(corpus)
dataloader = DataLoader(dataset, batch_size=1,
                shuffle=True)
    
decoder = DecoderRNN(256, dataset.tensor_size)
decoder.to(DEVICE)

criterion = torch.nn.BCEWithLogitsLoss(pos_weight=torch.full((dataset.tensor_size,),20, device=DEVICE)) # This loss combines a Sigmoid layer and the BCELoss in one single class
#Which is better for numerical stability.
learning_rate = 0.001
optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)

In [10]:
for i in range(10):
    train_epoch(dataloader, decoder, optimizer, criterion)
    torch.save({
            'epoch': i,
            'model_state_dict': decoder.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': 0,
            }, "model.pt")
    break
    
    
    
    
    

100%|██████████| 2443/2443 [08:31<00:00,  4.78it/s, loss=0.162] 


In [125]:
#Let's try to generate something
dataset = midiTextGeneratorDataset(corpus)

decoder.eval()
song = "4CL"
decoder_input = dataset.token_to_tensor(song).to(DEVICE)
finished = False
decoder_hidden = decoder.initHidden().to(DEVICE)
i = 0
prob = 0.7
previous = ""
repeated = 0
while not finished:
    decoder_output, decoder_hidden = decoder(
        decoder_input, decoder_hidden)
    
    decoder_output = torch.sigmoid(decoder_output)
    #print(decoder_output)
    decoder_output = (decoder_output > prob).type(dtype=torch.float32)
    #print(decoder_output)
    i+=1
    token = dataset.tensor_to_token(decoder_output)
    
    if previous == token:
        repeated +=1
        #print(repeated)
        if repeated > 20:
            #print("JEJ")
            #print(token)
            token = ''.join(random.choices(previous, k = min(len(previous), 5)))
            #print(token)
            repeated = 0
            previous = token
            decoder_input = dataset.token_to_tensor(token).to(DEVICE)
        else:
            decoder_input = decoder_output # i ths chord is input to the next iteration
            
    else:
        previous=token
        decoder_input = decoder_output # i ths chord is input to the next iteration
        
    if i == 200:
        break
    
    if i % 10 == 0:
        if prob == 0.8:
            prob = 0.7
        else:
            prob = 0.8
    song += " " + token 
    
    
    #decoder_input = decoder_output # i ths chord is input to the next iteration
    


In [126]:
print(song)

4CL L L L L L L L L L L L L L L L L L L L L L@ L@ EL4@ EL4X@ GE-LQ4X@ GE-LQ49X@ G>E-SLQ49_X ]&G>Ed-SL(Q ]&G2>Ed-SL( ]&G2>Ed/-SJ J2d2- ]&G2>Ed-SJL ]&G2>Ed/-SJ ]&G2>Ed/-SJ ]&G2>Ed/-SJ B]&G2>Ed/-S B]&G2>Ed/-S B]&G2>*Ed/- Bn]&Gk2>*Ed Bn]&Gk2>*Ed Bn]&Gk2>q*6 Bn]&Gk2>q*6 Bn]&Gk2U>q* Bn]&Gk2U>q* Bn]&Gk2U>q* Bn]&Gk2U>q* Bn]&Gk2U>q* Bn]&Gk2U>q* BnT]&Gk2U>q BnT]&Gk2U>q BnT]&Gk2U>q BnT]&Gk2U>q BnT]&Gk2U>q BnT]&Gk2U>q BnT]&Gk2U>q BnT]&Gk2U>q BnT]&Gk2U>q BnT]&Gk2U>q BnT]&Gk2U>q ]G>]B BnT]&Gk2U>q BnT]&Gk2U>q BnT]&Gk2U>q BnT]&Gk2U>q BnT]&Gk2U>q BnT]&Gk2U>q BnT]&Gk2U>q BnT]&Gk2U>q BnT]&Gk2U>q BnT]&Gk2U>q BnT]&Gk2U>q BnT]&Gk2U>q BnT]&Gk2U>q BnT]&Gk2U>q BnT]&Gk2U>q BnT]&Gk2U>q BnT]&Gk2U>q BnT]&Gk2U>q BnT]&Gk2U>q BnT]&Gk2U>q BnT]&Gk2U>q T&B>2 BnT]&Gk2U>q BnT]&Gk2U>q BnT]&Gk2U>q BnT]&Gk2U>q BnT]&Gk2U>q BnT]&Gk2U>q BnT]&Gk2U>q BnT]&Gk2U>q BnT]&Gk2U>q BnT]&Gk2U>q BnT]&Gk2U>q BnT]&Gk2U>q BnT]&Gk2U>q BnT]&Gk2U>q BnT]&Gk2U>q BnT]&Gk2U>q BnT]&Gk2U>q BnT]&Gk2U>q BnT]&Gk2U>q BnT]&Gk2U>q BnT]&Gk2U>q BnG2q BnT]&Gk2

In [127]:
c = converter.MidiTxtConverter()

track =c.string_to_multitrack(song.strip())

pypianoroll.write("lstm3.mid", track)